In [1]:
'''
@wcrd
This script processes a ttl file and builds a ag-grid formatted tree view data source
'''

import rdflib
from typing import Tuple, List
import json

In [2]:

## INPUT FILES
SWITCH_PATH = "../data/ontology/switch.ttl"
BRICK_PATH = "../data/ontology/brick.ttl"

### LOAD INTO GRAPH

In [3]:

## LOAD FILES INTO GRAPH
ds = rdflib.Dataset(default_union=True)
# Generic namespace I am going to use to name the graphs
ds_ns = rdflib.Namespace("https://_graph_.com#")
# lets load it with data from brick and switch.
# this will add graph if it doesn't exist, or return the existing one.
# could also do this independently using add_graph() method.
ds.graph(ds_ns['brick']).parse(BRICK_PATH, format='turtle');
ds.graph(ds_ns['switch']).parse(SWITCH_PATH, format="turtle");


KeyboardInterrupt: 

In [16]:
# generate namespaces
ds_g_ns = dict(ds.namespaces())
# check that graphs created.
contexts = ds.graphs()
print(f"Total DS entries: {len(ds)}")
print("Graphs created:")
for c in contexts:
    print(f"{c} ---- Entries: {len(ds.graph(c))}")

Total DS entries: 36789
Graphs created:
<https://_graph_.com#brick> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. ---- Entries: 34243
<https://_graph_.com#switch> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. ---- Entries: 2684
<urn:x-rdflib:default> a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']. ---- Entries: 0


### GENERATE TREE

In [17]:
# URI PROCESSOR
def uri_spliter(uri: rdflib.term.URIRef, ns: dict = {}) -> Tuple[str, str, str]:
    '''
    Given a URI, split into namespace and term.
    If a ns dict is given, return the prefix for that namespace as well
    (ns, term, prefix)
    '''
    try:
        chunks = uri.toPython().split("#")
        if len(chunks) == 1:
            # no # found, lets try spliting on last / (i.e. QUDT uses this system)
            chunks = uri.toPython().rsplit("/", 1)
        namespace = chunks[0]
        term = chunks[1]
    except IndexError:
        print(f"URI did not resolve into parts: {uri}")
        return (None,None,None)
    except:
        raise ValueError(f"Error parsing URI: {uri}")
    
    if ns:
        try:
            prefix = next(key for key, value in ns.items() if value.toPython() in [f"{namespace}#", f"{namespace}/"])
            return (namespace, term, prefix)
        except StopIteration:
            pass
    
    return (namespace, term, None)


In [18]:
# PARSE GRAPHS TO BUILD OUTPUT DATA
# get list of subjects to include in our reference file
subjects = ds.subjects()
# filter out BNodes
subjects = set(filter(lambda x: not isinstance(x, rdflib.term.BNode), subjects))
# filter out all 'Brick Tags'
subjects = list(filter(lambda x: f"{uri_spliter(x)[0]}#" != ds_g_ns['tag'].toPython(), subjects))
# filter out all true Shapes
subjects = list(filter(lambda x: f"{uri_spliter(x)[0]}#" != ds_g_ns['bsh'].toPython(), subjects))
# filter out Unit definitions
subjects = list(filter(lambda x: f"{uri_spliter(x)[0]}/" != ds_g_ns['unit'].toPython(), subjects))

print(len(subjects))

1715


In [19]:
#### Path Generating Functions

def get_class_heirarchy(class_uri, graph, base_level = 1, depth_limit=15, initial_path=[]):
    '''
    :base_level: how far down the class heirarchy to return as 'base'. base_level=0 is the root (for brick, this is simply 'Class')
    '''
    depth = 0
    # # get type of entity
    # entity_class = list(graph.objects(entity, rdflib.RDF.type))
    # if len(entity_class)==0:
    #     raise ValueError(f"No class found in graph for entity {entity}.")

    # Check Class exists
    if (class_uri, None, None) not in graph:
        raise ValueError(f"No class found in graph for class {class_uri}.")

    path = _climb_class_heirarchy(class_uri, graph, depth_limit, depth, initial_path)

    try:
        # return path[base_level]
        return path
    except IndexError:
        print(f"Requested base level {base_level} does not exist. Returning level 0 instead")
        return path[0]


def _climb_class_heirarchy(entity_class, graph, depth_limit, depth, path=[]) -> list:
    '''
    Path is used for storing the journey to the root class. Users can request base class at level {n}. Base of all classes is 'Class' which is level 0.
    '''
    # guard
    if depth >= depth_limit:
        raise RecursionError(f"Max depth of {depth_limit} reached. Increase depth_limit parameter if required.")

    # add current class to path
    path = [entity_class] + path

    # get parent
    parents = list(graph.objects(entity_class, rdflib.RDFS.subClassOf))
    # filter out BNodes - they are not useful to us here. We want explicit class definitions only.
    parents = list(filter(lambda x: not isinstance(x, rdflib.term.BNode), parents))
    
    # take each path and climb
    if len(parents)==0:
        # print("I'm done with this path. Returning: ")
        # print([path])
        return [path]
    else:
        paths = []
        for p in parents:
            new_paths = _climb_class_heirarchy(p, graph, depth_limit, depth+1, path)
            for new_path in new_paths:
                paths.append(new_path)
    return paths

In [20]:
# PATH FORMATTER (SIMPLIFIER)
def path_2_termString(path: List[rdflib.term.URIRef]) -> List[str]:
    '''
    Given a list of URIRefs, resolve to a list of term strings (no namespace)
    '''
    output = []
    for uri in path:
        _, term, _ = uri_spliter(uri)
        output.append(term)
    
    return output


In [21]:
# generate path data and data row
data = []
for idx, s in enumerate(subjects):
    namespace, term, prefix = uri_spliter(s, ds_g_ns)


    ### GOING TO SKIP THIS FOR NOW AND JUST PROCESS ENTITIES IF THEY ARE A CLASS DEFINTION OR OBJECT PROPERTY (relationships).
    ### TODO: Come back and update this to handle shapes, entityProperties, etc.

    # # Check type -> If Class, then get class path. If instance, get class, then get class path
    # # need to filter self out
    # s_class = list(filter(lambda x: x != s, ds.objects(s, rdflib.RDF.type)))
    # if len(s_class)==0:
    #     raise ValueError(f"No class found in graph for entity {s}.")
    # elif len(s_class) > 1:
    #     # raise ValueError(f"Too many types exist for the class. Please review: {s}.")
    #     print(f"Too many types exist for the class. SKIPPING. Please review: {s}.")
    #     continue
    # else:
    #     if s_class[0] != rdflib.OWL.Class:
    #         # Set this as first path item
    #         initial_path = s_class
    #     else: 
    #         initial_path = []

    ### BASIC VERSION
    # Get subject type(s) [filter self out]
    s_class = list(filter(lambda x: x != s, ds.objects(s, rdflib.RDF.type)))
    # filter down to only owl.class and owl.objectproperty, and sh:NodeShape so we can remove it -> these are what we will process
    s_class = list(filter(lambda x: x in [rdflib.OWL.Class, rdflib.OWL.ObjectProperty, rdflib.SH.NodeShape], s_class))
    
    if len(s_class)==0:
        print(f"No class found in graph for entity {s}. \nSKIPPING")
        continue

    if rdflib.SH.NodeShape in s_class:
        # OLD. We are excluding this now
        # # fixed path
        # paths = [[rdflib.SH.NodeShape, s]]

        # NEW
        # if the name term contains shape, skip it. Else treat it as a class.
        if "Shape" in term:
            print(f"Not progressing: {s}")
            continue
        else:
            s_class.remove(rdflib.SH.NodeShape)


    if s_class[0] != rdflib.OWL.Class:
        # If not a class then 
        paths = [[*s_class, s]]
    else:
        paths = get_class_heirarchy(s, ds)

    # Get description
    desc = list(ds.objects(s, rdflib.SKOS.definition))
    if len(desc) == 0: desc=""
    else: desc = desc[0]

    #if multiple paths, make multiple rows.
    for p_idx, path in enumerate(paths):
        data.append({
            "uri": s,
            "namespace": namespace,
            "prefix": prefix,
            "term": term,
            "desc": desc,
            "path": {
                "full": path,
                "agGridPath": path_2_termString(path)
            },
            "idx": f"{idx}.{p_idx}"
        })


No class found in graph for entity https://brickschema.org/schema/Brick#operationalStageCount. 
SKIPPING
Not progressing: https://brickschema.org/schema/Brick#TemperatureCoefficientPerDegreeCelsiusShape
No class found in graph for entity https://brickschema.org/schema/Brick#Differential_Dry_Bulb_Temperature. 
SKIPPING
No class found in graph for entity https://brickschema.org/schema/Brick#Dry_Bulb_Temperature. 
SKIPPING
No class found in graph for entity https://brickschema.org/schema/Brick#Apparent_Power. 
SKIPPING
No class found in graph for entity https://brickschema.org/schema/Brick#Torque. 
SKIPPING
No class found in graph for entity https://brickschema.org/schema/Brick#coolingCapacity. 
SKIPPING
No class found in graph for entity https://brickschema.org/schema/Brick#Electric_Energy. 
SKIPPING
No class found in graph for entity https://brickschema.org/schema/Brick#Flow_Loss. 
SKIPPING
No class found in graph for entity https://brickschema.org/schema/Brick#Real_Power. 
SKIPPING
No 

### CLEAN UP

In [22]:
### FINAL FILTERING
# Some split pathways end at sosa:ObservableProperty, sosa:FeatureOfInterest, skos:Concept. We are not interested in these and are going to remove objects that terminate there.
final_data = [item for item in data if item['path']['full'][0] not in [rdflib.SSN.ObservableProperty, rdflib.SSN.FeatureOfInterest, rdflib.SKOS.Concept]]

### MANUAL DATA INJECTION
Any objects that are needed for the website but that are not defined in provided ontology files. Generally just some high level rdf definition classes.

In [23]:
# Get last idx
# We can just manually write these for now as the idx range '0.x' is free.
# idx = int(float(final_data[-1]["idx"]))

# Add owl:ObjectProperty
# idx += 1
final_data.append({
    "uri": "http://www.w3.org/2002/07/owl#ObjectProperty",
    "namespace": "http://www.w3.org/2002/07/owl",
    "prefix": "owl",
    "term": "ObjectProperty",
    "desc": "A property axiom defines characteristics of a property. In its simplest form, a property axiom just defines the existence of a property. Object properties link individuals to individuals.",
    "path": {
      "full": [
        "http://www.w3.org/2002/07/owl#ObjectProperty"
      ],
      "agGridPath": [
        "ObjectProperty"
      ]
    },
    "idx": "0.1"
})

### EXPORT

In [24]:
with open("data.json", "w") as f:
    json.dump(final_data, f, indent=2)

## WORKING

In [44]:
s = subjects[13]
uri_spliter(subjects[13])
s_class = list(filter(lambda x: x != s, ds.objects(s, rdflib.RDF.type)))
s_class[0]

rdflib.term.URIRef('http://www.w3.org/ns/shacl#NodeShape')

In [45]:
rdflib.SH.NodeShape in s_class

True

In [56]:
rdflib.SSN.ObservableProperty

rdflib.term.URIRef('http://www.w3.org/ns/sosa/ObservableProperty')

In [39]:
"Shape" in uri_spliter(subjects[13])[1]

True

In [89]:
data[1174]

{'uri': rdflib.term.URIRef('https://brickschema.org/schema/Brick#Power'),
 'namespace': 'https://brickschema.org/schema/Brick',
 'prefix': 'brick',
 'term': 'Power',
 'path': {'full': [rdflib.term.URIRef('https://brickschema.org/schema/Brick#Power')],
  'agGridPath': ['Power']},
 'idx': '943.0'}

In [39]:
uri_spliter(rdflib.URIRef("test#1"), ds_g_ns)

('test', '1')

In [26]:
ds_g_ns['brick'].toPython()

'https://brickschema.org/schema/Brick#'

In [7]:
x1 = subjects[1]
x1

rdflib.term.URIRef('http://qudt.org/vocab/unit/NUM-PER-HR')

In [8]:
o = list(filter(lambda x: x != x1, list(ds.objects(x1, rdflib.RDF.type)))) 
#== rdflib.OWL.Class
o

[rdflib.term.URIRef('http://qudt.org/vocab/unit/Unit')]

In [44]:
rdflib.OWL.Class

rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class')

In [45]:
# filter down to only owl.class and owl.objectproperty
list(filter(lambda x: x in [rdflib.OWL.Class, rdflib.OWL.ObjectProperty], o))

[rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty')]

In [54]:
get_class_heirarchy(o[1], ds)

ValueError: No class found in graph for class http://www.w3.org/2002/07/owl#ObjectProperty.

In [16]:
ds.namespaces(x1)

TypeError: namespaces() takes 1 positional argument but 2 were given

In [30]:
x.toPython().split("#")[0] 

'https://brickschema.org/schema/Brick'

In [67]:
path_2_termString(data[0]['path'])

['Class', 'Location', 'Space', 'Room', 'Food_Service_Room', 'Concession']

In [64]:
data[0]['path']

[rdflib.term.URIRef('https://brickschema.org/schema/Brick#Class'),
 rdflib.term.URIRef('https://brickschema.org/schema/Brick#Location'),
 rdflib.term.URIRef('https://brickschema.org/schema/Brick#Space'),
 rdflib.term.URIRef('https://brickschema.org/schema/Brick#Room'),
 rdflib.term.URIRef('https://brickschema.org/schema/Brick#Food_Service_Room'),
 rdflib.term.URIRef('https://brickschema.org/schema/Brick#Concession')]

In [90]:
ds_g_ns

{'xml': rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace'),
 'rdf': rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#'),
 'rdfs': rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#'),
 'xsd': rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#'),
 'brick': rdflib.term.URIRef('https://brickschema.org/schema/Brick#'),
 'bsh': rdflib.term.URIRef('https://brickschema.org/schema/BrickShape#'),
 'dcterms1': rdflib.term.URIRef('http://purl.org/dc/terms#'),
 'owl': rdflib.term.URIRef('http://www.w3.org/2002/07/owl#'),
 'qudt': rdflib.term.URIRef('http://qudt.org/schema/qudt/'),
 'qudtqk': rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/'),
 'sdo': rdflib.term.URIRef('http://schema.org/'),
 'sh': rdflib.term.URIRef('http://www.w3.org/ns/shacl#'),
 'skos': rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#'),
 'sosa': rdflib.term.URIRef('http://www.w3.org/ns/sosa/'),
 'tag': rdflib.term.URIRef('https://brickschema.org/schema/BrickTag#'),
 'unit': rdfli

AttributeError: 'URIRef' object has no attribute 'namespace'

In [17]:
x1.split('#')

['http://qudt.org/vocab/unit/NUM-PER-HR']

In [18]:
x1.rsplit("/", 1)

['http://qudt.org/vocab/unit', 'NUM-PER-HR']

In [26]:
uri_spliter(x1)[0]+"/"

'http://qudt.org/vocab/unit/'

In [25]:
ds_g_ns['qudt']

rdflib.term.URIRef('http://qudt.org/schema/qudt/')